In [1]:
from freedom import freedom
from data import data
from analyze import analyze
import matplotlib.pyplot as plt
import seaborn as sb
import plotly as ply
import geopandas as geo
import pandas as pd
import numpy as np

In [2]:
data[0]

,Indicator Name,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Country Name,,,,,,,,,,,,,,,,,,,,,
Aruba,GDP per capita (current US$),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27546.899390,24631.434860,24271.940421,25324.720362,NaN,NaN,NaN,NaN,NaN,NaN
Afghanistan,GDP per capita (current US$),5.977733e+01,59.878153,58.492874,78.782758,82.208444,101.290471,137.899362,161.322000,129.506654,...,373.361116,445.893298,553.300289,603.537023,669.009051,638.612543,629.345250,569.577923,561.778746,585.850064
Angola,GDP per capita (current US$),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3868.579014,3347.844900,3531.416878,4299.008136,4539.467689,4804.633826,4707.578098,3683.552607,3308.772828,4170.312280
Albania,GDP per capita (current US$),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4370.539647,4114.136545,4094.358832,4437.178068,4247.614308,4413.081697,4578.666728,3952.830781,4131.872341,4537.862492
Andorra,GDP per capita (current US$),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,47785.659086,43339.379875,39736.354063,41098.766942,38391.080867,40619.711298,42294.994727,36038.267604,37231.815671,39146.548836
Arab World,GDP per capita (current US$),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,222.619525,...,6115.243747,5158.250506,5917.245724,6854.844526,7463.399723,7508.573301,7452.814677,6413.377161,6151.180927,6251.141575
United Arab Emirates,GDP per capita (current US$),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,45758.905536,33072.575166,35037.892673,40434.366350,42086.689953,43315.141492,44443.061514,39122.049705,38517.800386,40698.849336
Argentina,GDP per capita (current US$),NaN,NaN,1148.579957,845.077184,1166.317400,1272.010496,1266.344989,1057.755661,1136.517832,...,8953.359275,8161.306966,10276.260498,12726.908359,12969.707124,12976.636425,12245.256449,13698.293438,12654.354999,14401.974855
Armenia,GDP per capita (current US$),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4010.026997,2993.832531,3218.381655,3526.978143,3684.804810,3843.591213,3994.712355,3617.935746,3605.743117,3936.798320


In [ ]:
analyze(data, "GDP per capita (current US$)")

In [ ]:
df2 = freedom[[2017]].drop('Korea, Rep.', 0)

In [ ]:
df1 = np.log10(df2.join(analyze(data, 2017,["GDP per capita (current US$)"])))
df1.columns = ['x', 'y']
df1.plot('x', 'y', kind='scatter')
plt.show()

In [ ]:
def mmm(x=3,age=2017, ind="GDP per capita (current US$)"):
    df1 = analyze(data, age, [ind]).sort_values([ind])
    lmd1 = int(len(df1)/2)-int(x/2)
    df1.iloc[list(range(0,x))+list(range(lmd1, lmd1+x))+list(range(-x,0))].plot(kind="bar")
    plt.show()

In [ ]:
mmm()

In [ ]:
pd.DataFrame?

In [ ]:
x = 2
df1 = freedom[[2017]].drop('Korea, Rep.', 0).dropna().sort_values([2017])
lmd1 = int(len(df1)/2)
df1.iloc[list(range(0,x))+list(range(lmd1-int(x/2), lmd1+1+int(x/2)))+list(range(-x,0))].plot(kind="bar")
plt.show()

In [ ]:
df1 = analyze(data, 2017, ["GDP per capita (current US$)"])
df1.groupby(["GDP per capita (current US$)", pd.cut(df1.distances, 2)]).value.mean()



In [ ]:
def analyze(data, ageorcountryorind, indice=False, rm=True, log=False, disp=False):
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    from sklearn import linear_model
    countries = [i.replace("\n", "") for i in open("countries.txt")]
    
    if type(ageorcountryorind) == int:
        country = pd.DataFrame(data[0][str(ageorcountryorind)], columns=[data[0]['Indicator Name'][0]])
        for k,i in enumerate(data[1:]):
            country = country.join(pd.Series(i[str(ageorcountryorind)], name=data[k+1]["Indicator Name"][0]), how="outer")
        df = country.reindex(countries) if rm else country
    
    elif ageorcountryorind[0:2] == "c-":
        ages = pd.DataFrame(data[0].T[ageorcountryorind[2:]][4:], columns=[data[0]["Indicator Name"][0]])
        for k,i in enumerate(data[1:]):
            ages = ages.join(pd.Series(i.T[ageorcountryorind[2:]][4:], name=data[k+1]["Indicator Name"][0]), how="outer")
        df = ages
        
    else:
        for i in data:
            if i['Indicator Name'][0] == ageorcountryorind:
                saida = i.drop("Indicator Name", axis=1)
                break
        df = saida.reindex(countries) if rm else saida

    if indice==False:
        return df.dropna(how='all')
    elif log:
        df = np.log(df[indice].dropna()) 
        if disp:
            y = np.array(df[indice[0]]).reshape(-1, 1)
            x = np.array(df[indice[1]]).reshape(-1, 1)
            model = linear_model.LinearRegression().fit(x, y)
            plt.plot(x, model.predict(x), color='red', linewidth=2.)
            plt.scatter(y=y, x=x, color='blue', s=50, alpha=.5)
            plt.title('Reta ajustada log-log')
            plt.xlabel(indice[0])
            plt.ylabel(indice[1])
            plt.show()
        else:
            return df
    else:
        df = df[indice].dropna()
        if disp:
            y = df[indice[0]]
            x = df[indice[1]]
            plt.scatter(y=y, x=x, color='blue', s=50, alpha=.5)
            plt.title('Gráfico de dispersao')
            plt.xlabel(indice[0])
            plt.ylabel(indice[1])
            plt.show()
        else:
            return df
        